In [2]:
from deap import base, creator, tools
import random

In [ ]:
# Problem constants
# Length of bit string to be optimized
ONE_MAX_LENGTH = 100

In [ ]:
# Genetic Algorithm constants
POPULATION_SIZE = 200
# Probability for crossover
P_CROSSOVER = 0.9
# Probability for mutating an individual
P_MUTATION = 0.1
MAX_GENERATIONS = 50

In [ ]:
# Set the random seed
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [5]:
toolbox = base.Toolbox()
# Create an operator defining the alias of `random.randint(0, 1)` as `zeroOrOne`
toolbox.register("zeroOrOne", random.randint, 0, 1)

# Populate a `list` with `initRepeat`
# randomList = tools.initRepeat(list, toolbox.zeroOrOne, 30)
# print(randomList)

[0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1]


In [6]:
# Define the fitness strategy by extending the `base.Fitness` class
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

In [7]:
# Create the `Individual` class based on `list`
creator.create("Individual", list, fitness=creator.FitnessMax)

In [ ]:
# Create the `individualCreator` operator to fill up an `Individual` instance
toolbox.register("individualCreator", tools.initRepeat, creator.Individual, toolbox.zeroOrOne, ONE_MAX_LENGTH)

In [ ]:
# Create the population operator to generate a list of individuals
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)

In [ ]:
# Fitness Calculation
# Compute the number of `1`s in the individual
def oneMaxFitness(individual):
    # Return a Tuple
    return sum(individual)

toolbox.register("evaluate", oneMaxFitness)

In [ ]:
# Genetic Operators
# Tournament selection with tournament size of `3`
toolbox.register("select", tool.selTournament, tournsize=3)